In [1]:
import os
import sys
import json
import pickle

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import pandas as pd

import functools

import numpy as np
import math
import random
import scipy

import plotting
import utils
import yaml

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
from data_loader import BiosDataset, JigsawBias, CustomDataset, CrowSPairs, StereoSet, TwitterAAE, SBICDataset

2025-05-08 14:36:29.585617: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746707789.598867  435370 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746707789.602777  435370 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746707789.612622  435370 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746707789.612638  435370 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746707789.612640  435370 computation_placer.cc:177] computation placer alr

In [2]:
CONFIG_FILE = 'configs/esann25/experiment_config.json'

with open(CONFIG_FILE, 'r') as f:
    config = json.load(f)

openai_models = config["openai_models"]
huggingface_models = config["huggingface_models"]
models = huggingface_models + openai_models
emb_dir = '../%s' % config['embedding_dir']

In [3]:
def get_dataset_stats(g_train, g_test, groups):
    stats = {'train': {group: 0 for group in groups}, 'test': {group: 0 for group in groups}}
    df = pd.DataFrame(stats)
    
    if g_test.ndim == 1:
        # single-label
        if len(g_train) > 0:
            df.loc[groups[1], 'train'] = np.sum(g_train)
            df.loc[groups[0], 'train'] = len(g_train)-np.sum(g_train)
        df.loc[groups[1], 'test'] = np.sum(g_test)
        df.loc[groups[0], 'test'] = len(g_test)-np.sum(g_test)
        
        df.loc['total', :] = df.loc[groups, :].sum(axis=0)
    else:
        # multi-label
        for i, group in enumerate(groups):
            if len(g_train) > 0:
                df.loc[group, 'train'] = np.sum(g_train[:,i])
            df.loc[group, 'test'] = np.sum(g_test[:,i])
        df.loc['total', 'train'] = len(g_train)
        df.loc['total', 'test'] = len(g_test)
    return df


def print_dataset_stats(ds: CustomDataset, as_percentage=True):
    # protected groups
    group_dist = {split: {group: 0 for group in ds.group_names} for split in ds.splits}
    group_df = pd.DataFrame(group_dist)

    for split in ds.splits:
        for i, group in enumerate(ds.group_names):
            group_df.loc[group, split] = np.sum(ds.protected_groups[split][:,i])
        total = ds.n_samples[split]
        if as_percentage:
            group_df.loc[:,split] *= 100/total
        group_df.loc['total #', split] = total

    if as_percentage:
        group_df.columns = [(col+' [%]') for col in group_df.columns]
    
    print("protected groups:")
    print(group_df)
    print()

    # classes
    if not ds.unlabeled:
        lbl_dist = {split: {cls: 0 for cls in ds.class_names} for split in ds.splits}
        lbl_df = pd.DataFrame(lbl_dist)
    
        for split in ds.splits:
            if ds.multi_label:
                for i, cls in enumerate(ds.class_names):
                    lbl_df.loc[cls, split] = np.sum(ds.labels[split][:,i])
            else: # single-label
                for i, cls in enumerate(ds.class_names):
                    lbl_df.loc[cls, split] = np.sum(ds.labels[split] == i)
            total = ds.n_samples[split]
            if as_percentage:
                lbl_df.loc[:,split] *= 100/total
            lbl_df.loc['total #', split] = total

        if as_percentage:
            lbl_df.columns = [(col+' [%]') for col in lbl_df.columns]
    
        print("class label:")
        print(lbl_df)

In [4]:
stereoset = StereoSet()
print_dataset_stats(stereoset)

Using the latest cached version of the dataset since McGill-NLP/stereoset couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'intersentence' at /home/saschroeder/.cache/huggingface/datasets/McGill-NLP___stereoset/intersentence/0.0.0/bf6e7ce50491784d094fb7afe60a70ecccb89035 (last modified on Wed May  7 13:03:29 2025).


load Stereoset with option both
Load inter- and intrasentence samples and merge them to one dataset


Using the latest cached version of the dataset since McGill-NLP/stereoset couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'intrasentence' at /home/saschroeder/.cache/huggingface/datasets/McGill-NLP___stereoset/intrasentence/0.0.0/bf6e7ce50491784d094fb7afe60a70ecccb89035 (last modified on Wed May  7 13:03:32 2025).


compute class weights for split val 
protected groups:
                       val [%]
european/white       11.153149
african/black        14.644912
hispanic              5.005123
south east asian      5.359817
middle east          13.210373
male                 26.026641
female               15.448885
christian             1.505478
muslim                1.891700
hindu                 1.166548
jewish                0.173406
total #           12687.000000

class label:
                      val [%]
anti-stereotype     33.333333
stereotype          33.333333
unrelated           33.333333
total #          12687.000000


/tmp/ipykernel_433210/2816108568.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[11.15314889 14.64491211  5.00512335  5.35981714 13.21037282 26.02664144
 15.44888469  1.50547805  1.89170017  1.16654844  0.17340585]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  group_df.loc[:,split] *= 100/total
/tmp/ipykernel_433210/2816108568.py:59: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[33.33333333 33.33333333 33.33333333]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  lbl_df.loc[:,split] *= 100/total


In [38]:
csp = CrowSPairs()
print_dataset_stats(csp)

load crowspairs
protected groups:
                                       test [%]
black                                 10.444297
white                                 12.665782
asian                                  1.094164
poor                                   4.244032
rich                                   2.751989
male                                  43.103448
female                                23.309019
physical disability, sickness          1.326260
mental_disability_illness              1.094164
foreigner, immigrant, undocumented     0.530504
canadian                               0.397878
us american                            5.935013
mexican                                1.591512
chinese                                0.961538
italian                                0.928382
british                                0.464191
russian                                0.298408
indian                                 0.629973
homosexual                             2.055703
hetero

/tmp/ipykernel_122704/2816108568.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[10.44429708 12.66578249  1.09416446  4.24403183  2.75198939 43.10344828
 23.30901857  1.32625995  1.09416446  0.53050398  0.39787798  5.93501326
  1.59151194  0.96153846  0.92838196  0.46419098  0.29840849  0.62997347
  2.05570292  1.92307692  1.42572944  0.82891247  0.96153846  1.06100796
  0.59681698  3.01724138  1.29310345  1.55835544  2.81830239  3.28249337]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  group_df.loc[:,split] *= 100/total
/tmp/ipykernel_122704/2816108568.py:59: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[50. 50.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  lbl_df.loc[:,split] *= 100/total


In [39]:
taae = TwitterAAE()
print_dataset_stats(taae)

load twitterAAE
protected groups:
         test [%]
aa           50.0
white        50.0
total #  100000.0



In [40]:
bios = BiosDataset(option='unsupervised')
print_dataset_stats(bios)

load BIOS with option unsupervised
protected groups:
             train [%]      test [%]       dev [%]
male         53.899751     53.898798     53.897382
female       46.100249     46.101202     46.102618
total #  257478.000000  99069.000000  39642.000000

class label:
                       train [%]      test [%]       dev [%]
accountant              1.421481      1.422241      1.422733
architect               2.550898      2.551757      2.552848
attorney                8.221673      8.219524      8.218556
chiropractor            0.669960      0.670240      0.671005
comedian                0.708410      0.709606      0.711367
composer                1.412548      1.414166      1.415166
dentist                 3.681480      3.681273      3.680440
dietitian               0.996978      0.998294      0.998941
dj                      0.374401      0.375496      0.378387
filmmaker               1.765199      1.766446      1.768327
interior_designer       0.368575      0.369439      0.3682

/tmp/ipykernel_122704/2816108568.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[53.89975066 46.10024934]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  group_df.loc[:,split] *= 100/total
/tmp/ipykernel_122704/2816108568.py:59: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 1.42148067  2.55089755  8.22167331  0.66996015  0.70841004  1.41254787
  3.68147958  0.99697838  0.37440092  1.76519936  0.36857518  5.03343975
  1.89025859  4.78332129  1.95162305  0.44508657  0.63617086  0.36041914
  6.12596028 10.3496221   1.77024833 29.80759521  4.63923131  0.35381664
  1.74461507  3.42903083  4.09005818  0.41789978]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  lbl_df.loc[:,split] *= 100/total


In [41]:
local_dir = '../../../data/bios_huggingface_merge.pkl'
bios = BiosDataset(option='supervised', local_dir=local_dir)
print_dataset_stats(bios)

load BIOS with option supervised
protected groups:
           train [%]  test [%]      dev [%]
male       57.346444     56.12    57.839388
female     42.653556     43.88    42.160612
total #  7017.000000   2500.00  1046.000000

class label:
                train [%]  test [%]      dev [%]
architect        3.648283      3.40     3.154876
surgeon         10.089782     10.16    10.038241
dentist          1.667379      1.12     1.720841
teacher         12.042183     13.08    12.141491
psychologist     9.633747      8.68     9.177820
nurse            6.156477      6.32     5.736138
photographer    17.030070     16.72    17.304015
physician       11.742910     10.80    10.420650
attorney        13.025509     14.48    15.105163
journalist      17.300841     18.04    17.304015
total #       7017.000000   2500.00  1046.000000


/tmp/ipykernel_122704/2816108568.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[57.34644435 42.65355565]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  group_df.loc[:,split] *= 100/total
/tmp/ipykernel_122704/2816108568.py:59: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 3.64828274 10.08978196  1.66737922 12.04218327  9.63374662  6.15647713
 17.03006983 11.74291008 13.02550948 17.30084082]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  lbl_df.loc[:,split] *= 100/total


In [5]:
local_dir = '../../../data/jigsaw_bias'
jigsaw = JigsawBias(option='multi-class', local_dir=local_dir)
print_dataset_stats(jigsaw)

load JigsawBias with option: multi-class
protected groups:
                                         train [%]       dev [%]      test [%]
female                                   11.159822     10.230500     10.080623
male                                      8.363369      8.040201      7.380553
transgender                               0.392480      0.426043      0.384178
white                                     3.797435      3.561285      3.181646
asian                                     0.674060      0.704610      0.589795
black                                     2.069832      1.878960      1.834316
latino                                    0.245191      0.245794      0.297603
atheist                                   0.265407      0.507975      0.611439
buddhist                                  0.101947      0.071007      0.097397
christian                                 7.610755      7.324667      7.375142
hindu                                     0.102235      0.081931      0.

/tmp/ipykernel_123613/2816108568.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1.11598221e+01 8.36336857e+00 3.92479640e-01 3.79743545e+00
 6.74059955e-01 2.06983192e+00 2.45191475e-01 2.65407497e-01
 1.01946514e-01 7.61075492e+00 1.02235315e-01 1.48327846e+00
 3.58949922e+00 1.61728181e-01 2.71472304e-02 1.56731936e+00
 4.62080518e-03 4.04320453e-03 7.16802403e-01]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  group_df.loc[:,split] *= 100/total
/tmp/ipykernel_123613/2816108568.py:59: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2.92641368 0.15190897 0.38930284 2.55328366 0.06382487 0.06498007]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  lbl_df.loc[:,split] *= 100/total


In [4]:
local_dir = '../../../data/jigsaw_bias'
jigsaw = JigsawBias(option='single-class', local_dir=local_dir)
print_dataset_stats(jigsaw)

load JigsawBias with option: single-class
protected groups:
                                         train [%]       dev [%]      test [%]
female                                   11.312843     10.346472     10.161748
male                                      8.495626      8.195592      7.488709
transgender                               0.413703      0.455605      0.393866
white                                     4.237590      4.111040      3.523790
asian                                     0.678955      0.704598      0.577670
black                                     2.380265      2.230345      2.153135
latino                                    0.256289      0.254291      0.309841
atheist                                   0.274775      0.519178      0.614431
buddhist                                  0.103356      0.074168      0.099779
christian                                 7.649733      7.347955      7.420439
hindu                                     0.102516      0.084764      0

/tmp/ipykernel_130334/2816108568.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1.13128433e+01 8.49562628e+00 4.13703472e-01 4.23758960e+00
 6.78955462e-01 2.38026548e+00 2.56288881e-01 2.74775292e-01
 1.03355844e-01 7.64973293e+00 1.02515552e-01 1.53941387e+00
 3.86478031e+00 1.77861682e-01 2.82898109e-02 1.78393867e+00
 4.48155420e-03 3.92135993e-03 7.20970032e-01]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  group_df.loc[:,split] *= 100/total
/tmp/ipykernel_130334/2816108568.py:59: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[94.81680247  5.18319753]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  lbl_df.loc[:,split] *= 100/total


In [4]:
local_dir='../../../data/filtered_sbic_minority_overview.csv'
sbic = SBICDataset(local_dir=local_dir)
print_dataset_stats(sbic)

Using the latest cached version of the module from /home/saschroeder/.cache/huggingface/modules/datasets_modules/datasets/allenai--social_bias_frames/79706db13a32c7f9614b997cc4326cbda14e6d3968892a3f0e76c4a970e7e510 (last modified on Thu Apr 10 17:11:43 2025) since it couldn't be found locally at allenai/social_bias_frames, or remotely on the Hugging Face Hub.


load SBIC with local file: ../../../data/filtered_sbic_minority_overview.csv


Using the latest cached version of the module from /home/saschroeder/.cache/huggingface/modules/datasets_modules/datasets/allenai--social_bias_frames/79706db13a32c7f9614b997cc4326cbda14e6d3968892a3f0e76c4a970e7e510 (last modified on Thu Apr 10 17:11:43 2025) since it couldn't be found locally at allenai/social_bias_frames, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /home/saschroeder/.cache/huggingface/modules/datasets_modules/datasets/allenai--social_bias_frames/79706db13a32c7f9614b997cc4326cbda14e6d3968892a3f0e76c4a970e7e510 (last modified on Thu Apr 10 17:11:43 2025) since it couldn't be found locally at allenai/social_bias_frames, or remotely on the Hugging Face Hub.


compute class weights for split train 
compute class weights for split test 
no samples found for label cancer
no samples found for label orphans
compute class weights for split val 
protected groups:
                                       train [%]     test [%]      val [%]
white                                   0.333955     0.361318     0.277778
black                                   8.312693    14.410202    13.611111
asian                                   1.775527     2.146652     2.371795
non-white                               8.479670    14.495218    13.717949
latin-american                          0.111318     0.233794     0.235043
hispanic                                0.976818     1.530287     1.239316
mixed race                              0.016698     0.021254     0.064103
middle eastern                          0.484235     0.488842     0.555556
male                                    1.730999     1.806589     2.029915
female                                  8.802494 

/tmp/ipykernel_435370/2816108568.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.33395486 8.31269307 1.77552667 8.4796705  0.11131829 0.97681797
 0.01669774 0.48423455 1.73099936 8.80249353 0.02782957 0.43414132
 0.24768319 1.49166504 1.80892216 0.43414132 2.76904238 1.45270364
 4.62249186 0.4675368  0.41466062 0.71521999 0.98516684 0.70687112
 0.0194807  0.05565914 0.34230373 0.38404809 0.41187766 0.12523307
 0.13079899 0.22263657 0.90724404 0.26716389 0.71800295 0.18924109
 0.10296942 0.0194807  0.07235689 4.92583419 0.95455431 0.34230373
 0.389614   0.88219742 0.92115882 0.12801603]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  group_df.loc[:,split] *= 100/total
/tmp/ipykernel_435370/2816108568.py:59: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[45.60710211 47.68875407  8.79414466]' has d